In [5]:
import pandas as pd
import os
import geopandas as gpd
import re # regular expression
#import pyproj

In [6]:
# GLOBAL VARIABLES

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/WFv901_Segments_20240226_GeoKeys_AADTs_Factors.shp'

# filter by PLANAREA in segments shapefile
#filterPlanArea = ['WFRC','MAG'] # must be an array... if only single item, the still include []

# Get AADT and Previous Forecasts from Segment Shapefile

In [7]:
# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
display(gdfSegments[gdfSegments['SEGID'].duplicated(keep=False)])
display(gdfSegments)

# show columns
#print(gdfSegments.columns.tolist())

,Id,SEGID,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,AADTSTN,AADT2022,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,geometry


,Id,SEGID,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,AADTSTN,AADT2022,...,FAC_SUM,FAC_SUN,FAC_THU,FAC_TUE,FAC_WDAVG,FAC_WEAVG,FAC_WED,FAC_WEMAX,FAC_WIN,geometry
0,0,0006_141.0,141.035,146.868,5.843301,49.0,1.0,MAG,049-0005,1741.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (405824.110 4423860.330, 406035.300..."
1,0,0006_146.9,146.868,149.902,3.039023,49.0,1.0,MAG,049-0005,1741.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (413442.550 4422753.728, 413459.600..."
2,0,0006_149.9,149.902,150.580,0.677983,49.0,1.0,MAG,049-0010,2803.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (418330.800 4422866.000, 418629.100..."
3,0,0006_150.6,150.580,152.555,1.973104,49.0,1.0,MAG,049-0010,2803.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (419421.880 4422872.296, 419780.300..."
4,0,0006_152.6,152.555,152.871,0.316275,49.0,1.0,MAG,049-0011,2363.0,...,1.0914,0.8819,1.0120,0.9628,0.9840,0.9734,0.9773,1.0649,0.8495,"LINESTRING (422596.900 4422889.295, 422598.500..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,0,WFRC_8469,0.000,0.000,0.567664,35.0,1.0,WFRC,0,0.0,...,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370,"LINESTRING (407213.374 4504030.989, 407034.934..."
4976,0,WFRC_8470,0.000,0.000,0.493148,35.0,1.0,WFRC,0,0.0,...,1.0575,0.6429,1.1132,1.0744,1.0918,0.7585,1.0878,0.8742,0.8994,"LINESTRING (407723.200 4503422.753, 407213.374..."
4977,0,WFRC_8471,0.000,0.000,0.405891,35.0,1.0,WFRC,0,0.0,...,1.0283,0.6061,1.1071,1.0813,1.0924,0.7653,1.0887,0.9245,0.9370,"LINESTRING (415734.595 4481608.605, 415755.740..."
4978,0,WFRC_8472,0.000,0.000,0.253734,35.0,1.0,WFRC,0,0.0,...,1.0135,0.6293,1.1063,1.0851,1.0946,0.7583,1.0925,0.8873,0.9653,"LINESTRING (424308.500 4489341.200, 424304.312..."


In [8]:
# filter segments by PLANAREA
_df = gdfSegments#[gdfSegments['PLANAREA'].isin(filterPlanArea)].copy()

# get columns with AADT at beginning of name
aadt_columns = [col for col in _df.columns if col.startswith('AADT') and col != 'AADTSTN']

# get dataframe for just segids and aadt columns
_df = _df[['SEGID'] + aadt_columns]

# melt to make long
_df = _df.melt(id_vars='SEGID', var_name='YEAR', value_name='AADT')

# filter out zero volumes
_df = _df[_df['AADT']>0]

# get YEAR from string
_df['YEAR'] = _df['YEAR'].str[-4:].astype(int)

# set AADT to int
_df['AADT'] = _df['AADT'].astype(int)

# set source
_df['SOURCE'] = fnSegmentsShapefile.split('/')[-1]

dfAadtFromSegments = _df

dfAadtFromSegments

,SEGID,YEAR,AADT,SOURCE
0,0006_141.0,2022,1741,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
1,0006_146.9,2022,1741,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
2,0006_149.9,2022,2803,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
3,0006_150.6,2022,2803,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
4,0006_152.6,2022,2363,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
...,...,...,...,...
207371,2899_002.4,1981,2635,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
207372,2899_003.5,1981,2635,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
207373,2899_004.1,1981,850,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
209108,WFRC_8419,1981,15075,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...


# Wrap up and export

In [9]:
dfAadt = pd.concat([dfAadtFromSegments])
dfAadt = dfAadt.sort_values(by=['SEGID','SOURCE','YEAR'])
dfAadt = dfAadt.drop_duplicates()
dfAadt

,SEGID,YEAR,AADT,SOURCE
204180,0006_141.0,1981,975,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
199200,0006_141.0,1982,1000,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
194220,0006_141.0,1983,965,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
189240,0006_141.0,1984,1240,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
184260,0006_141.0,1985,1340,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
...,...,...,...,...
24893,WFRC_8467,2018,26161,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
19913,WFRC_8467,2019,26632,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
14933,WFRC_8467,2020,23782,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
9953,WFRC_8467,2021,25780,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...


In [10]:
# CHECK NON-NUMERIC SEGIDs!!
dfAadt['SEGID'][dfAadt['SEGID'].str.match(r'[^\d]')].drop_duplicates()


28895      MAG_6017
9018       MAG_6067
9020       MAG_6069
9087       MAG_6139
29008      MAG_6140
64309     WFRC_8004
39414     WFRC_8010
99354     WFRC_8220
64533     WFRC_8262
209108    WFRC_8419
209153    WFRC_8467
Name: SEGID, dtype: object

In [11]:
dfAadt.rename(columns={'AADT':'obsAADT'})

# export AADT
dfAadt.to_csv('intermediate/aadt.csv',index=False)

# export AADT sources
dfAadt[['SOURCE']].drop_duplicates().to_csv('intermediate/aadt-sources.csv',index=False)

## export Forecasts
#dfForecastsFromSegments.to_csv('intermediate/previous-forecasts.csv',index=False)

## export AADT sources
#dfForecastsFromSegments[['SOURCE']].drop_duplicates().to_csv('intermediate/previous-forecasts-sources.csv',index=False)

In [12]:
dfAadt[dfAadt['SEGID']=='2330_005.2']

,SEGID,YEAR,AADT,SOURCE
22750,2330_005.2,2018,4284,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
17770,2330_005.2,2019,4361,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
12790,2330_005.2,2020,3894,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
7810,2330_005.2,2021,4221,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...
2830,2330_005.2,2022,4263,WFv901_Segments_20240226_GeoKeys_AADTs_Factors...


# create shapefile

In [13]:
#_df = dfAadt
#_df['YEARCOL'] = "AADT" + _df['YEAR'].astype(str)
#_df['AADT'] = _df['AADT'].astype(int)
#_df2 = _df.groupby(['SEGID','YEARCOL'],as_index=False).agg(cntRows=('YEAR','count'))
#_df2 = _df.pivot(index="SEGID",columns=('YEARCOL'), values='AADT')
#
#_df2.fillna(0,inplace=True)
#
#_df3 = pd.DataFrame.merge(gdfSegments[['SEGID','geometry']],_df2, on="SEGID", how="left")
#_df3.fillna(0,inplace=True)
#
#
## Identify numeric columns
#numeric_cols = _df3.select_dtypes(include=['float64']).columns
#
## Convert numeric columns to integer type
#for col in numeric_cols:
#    _df3[col] = _df3[col].astype(int)
#
#from datetime import datetime
#
## Generate a filename with the current timestamp in the specified folder
#timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#
#_df3.to_file('results/Segments_State_' + timestamp + '_Draft_AADTOnly.shp')